# Setup COCO API


In [1]:
!pip install Cython
!pip install -q "tqdm>=4.36.1"
!pip3 install xgboost
!pip3 install scikit-image scikit-learn graphviz

In [2]:
!git clone https://github.com/waleedka/coco
!pip install -U setuptools
!pip install -U wheel
!make install -C coco/PythonAPI

fatal: destination path 'coco' already exists and is not an empty directory.
Requirement already up-to-date: setuptools in /opt/conda/lib/python3.7/site-packages (50.3.2)
Requirement already up-to-date: wheel in /opt/conda/lib/python3.7/site-packages (0.35.1)
make: Entering directory '/home/jupyter/modified-FCOS/DTL/coco/PythonAPI'
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
building 'pycocotools._mask' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
creating build/common
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/lib/python3.7/site-packages/numpy/core/include -I../common -I/opt/conda/include/python3.7m -c pycocotools/_mask.c -o build/temp.linux-x86_64-3.7/pycocotools/_mask.o -Wno-cpp -Wno-unused-function -std=c99
^C
interrupted
Makefile:7: recipe for target 'install' 

In [3]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip ./annotations_trainval2017.zip

--2020-11-06 01:44:33--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.88.163
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.88.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

      annotations_t   0%[                    ] 223.26K   263KB/s               ^C
Archive:  ./annotations_trainval2017.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./annotations_trainval2017.zip or
        ./annotations_trainval2017.zip.zip, and cannot find ./annotations_trainval2017.zip.ZIP, period.


# Import needed and Load COCO categories

In [5]:
from pycocotools.coco import COCO
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.util import crop
from skimage import io, color
from math import floor, ceil
from PIL import Image
from joblib import dump, load
from tqdm.notebook import tqdm
from os import path
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import graphviz 
import json
import pandas as pd

In [6]:
def split_integer(num):
    return (floor(num/2), ceil(num/2))

def create_dataset_id(coco):
    catIds = coco.getCatIds(catNms=['dog','cat'])
    img_catdog = coco.getImgIds(catIds=catIds)
    img_dog = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=['dog']))) - set(img_catdog))
    img_cat = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=['cat']))) - set(img_catdog))
    print("Dog Image Total: ",len(img_dog),"\nCat Image Total: ", len(img_cat))
    return img_dog + img_cat, np.array(len(img_dog) * [1] + len(img_cat) * [0])

def load_resize_crop_img(img_url, target_width=64, rgb = True):
    I = io.imread(img_url)
    factor = target_width/min(I.shape[0:2])
    resize_target = (round(I.shape[0] * factor), round(I.shape[1] * factor))
    resized_image = (resize(I, resize_target, anti_aliasing=False) * 255).astype(np.uint8)
    crop_width = (split_integer(resize_target[0] - target_width), split_integer(resize_target[1] - target_width), (0,0))
    if rgb:
        cropped_image = crop(color.gray2rgb(resized_image), crop_width)
    else:
        cropped_image = crop(color.rgb2gray(resized_image), crop_width)
    return cropped_image

def flatten_img(img):
    return img.flatten()

In [7]:
annotation = {"train":"annotations/instances_train2017.json", "val":"annotations/instances_val2017.json"}
train_path="annotations/instances_train2017.json"
val_path="annotations/instances_val2017.json"



*   Gabungin Data training dan validasi,
*   Cari tau kalau di DTL, Random Forest sama XGBoost ada train/val/test ga?
- Apakah urutan pengaruh untuk hasil?
- Bikin fungsi buat evaluate hasilnya ke set tertentu. Coba cari tau evaluation metrics apa yang relevan buat ini  di SKLearn



In [8]:
def get_coco_id(annotation_file, cat1 = "dog", cat2 = "cat", n = None):
    coco = COCO(annotation_file)
    catIds = coco.getCatIds(catNms=[cat1,cat2])
    img_combine = coco.getImgIds(catIds=catIds)
    img_cat1 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat1]))) - set(img_combine))
    img_cat2 = list(set(coco.getImgIds(catIds=coco.getCatIds(catNms=[cat2]))) - set(img_combine))
    if n is None:
        return img_cat1 + img_cat2, np.array(len(img_cat1) * [1] + len(img_cat2) * [0])
    return img_cat1[:n] + img_cat2[:n], np.array(len(img_cat1[:n]) * [1] + len(img_cat2[:n]) * [0])

In [9]:
X_train, y_train = get_coco_id(annotation["train"])
X_val, y_val = get_coco_id(annotation["val"])

loading annotations into memory...
Done (t=21.85s)
creating index...
index created!
loading annotations into memory...
Done (t=0.65s)
creating index...
index created!


In [16]:
coco = COCO(annotation["train"])
for x in tqdm(X_train):
    fn = "train/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=18.34s)
creating index...
index created!


In [17]:
coco = COCO(annotation["val"])
for x in tqdm(X_val):
    fn = "val/{}.png".format(x)
    if path.exists(fn):
#         print(f"skipping {x}")
        continue
    I = io.imread(coco.loadImgs(x)[0]['coco_url'])
    io.imsave(fn,I)

loading annotations into memory...
Done (t=1.89s)
creating index...
index created!


In [10]:
# coco = COCO(annotation["train"])
X_loaded = []
for x in tqdm(X_train):
    try : 
        fn = "train/{}.png".format(x)
        X_loaded += [load_resize_crop_img(fn).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded))
X_loaded = np.array(X_loaded)
# df_train = pd.Dataframe({"X":X_loaded,"y":y_train})
print("Converted to ndarray with shape: ",X_loaded.shape, y_train.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  4000
Converted to ndarray with shape:  (4000, 12288) (4000,)


In [11]:
# coco = COCO(annotation["val"])
X_loaded_val = []
for x in tqdm(X_val):
    try : 
        fn = "val/{}.png".format(x)
        X_loaded_val += [load_resize_crop_img(fn).flatten()]
    except :
        print(x)
print("Finished with length: ",len(X_loaded_val))
X_loaded_val = np.array(X_loaded_val)
print("Converted to ndarray with shape: ",X_loaded_val.shape, y_val.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Pass-through of possibly RGB images in gray2rgb is deprecated. In version 0.19, input arrays will always be considered grayscale, even if the last dimension has length 3 or 4. To prevent this warning and ensure compatibility with future versions, detect RGB images outside of this function.



Finished with length:  337
Converted to ndarray with shape:  (337, 12288) (337,)


In [12]:
test_fold = np.concatenate([
    # The training data.
    np.full(X_loaded.shape[0], -1, dtype=np.int8),
    # The development data.
    np.zeros(X_loaded_val.shape[0], dtype=np.int8)
])
cv = PredefinedSplit(test_fold)

X = np.concatenate([X_loaded, X_loaded_val])
y = np.concatenate([y_train, y_val])

# dtrain = xgb.DMatrix(X_loaded, label=y_train)
# dval = xgb.DMatrix(X_loaded_val, label=y_val)

In [13]:
base = DecisionTreeClassifier()
grid = {
"max_depth" : [10, 100, 1000],
"criterion" : ["gini", "entropy"]
}
model = GridSearchCV(base, grid, cv=cv,n_jobs=4, verbose=100)

In [14]:
model = model.fit(X,y)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   53.2s
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:  1.1min remaining:  2.3min
[Parallel(n_jobs=4)]: Done   3 out of   6 | elapsed:  1.2min remaining:  1.2min
[Parallel(n_jobs=4)]: Done   4 out of   6 | elapsed:  1.4min remaining:   41.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  2.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  2.3min finished


In [15]:
print(model.cv_results_)

{'mean_fit_time': array([52.05243278, 67.40027165, 68.80992842, 80.66512585, 76.65196061,
       72.17801356]), 'std_fit_time': array([0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.01449633, 0.01288176, 0.01296735, 0.01156735, 0.0100491 ,
       0.00946951]), 'std_score_time': array([0., 0., 0., 0., 0., 0.]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[10, 100, 1000, 10, 100, 1000],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'criterion': 'gini', 'max_depth': 10}, {'criterion': 'gini', 'max_depth': 100}, {'criterion': 'gini', 'max_depth': 1000}, {'criterion': 'entropy', 'max_depth': 10}, {'criterion': 'entropy', 'max_depth': 100}, {'criterion': 'entropy', 'max_depth': 1000}], 'split

In [16]:
dump(model, 'gs_dtl.pkl')

['gs_dtl.pkl']

In [ ]:
%reset_selective "^model$"

In [17]:
rf_base = xgb.XGBRFClassifier()
rf_grid = {
"max_depth" : [100, 1000],
"learning_rate" : [0.1, 1],
"n_estimators" : [50, 100, 200]
}
rf_model = GridSearchCV(rf_base, rf_grid, cv=cv,n_jobs=4, verbose=100)

In [18]:
rf_model = rf_model.fit(X, y)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 12.9min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed: 13.1min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed: 26.1min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed: 38.9min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 39.1min
[Parallel(n_jobs=4)]: Done   6 out of  12 | elapsed: 51.8min remaining: 51.8min
[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 64.8min remaining: 46.3min
[Parallel(n_jobs=4)]: Done   8 out of  12 | elapsed: 64.8min remaining: 32.4min
[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 65.4min remaining: 21.8min
[Parallel(n_jobs=4)]: Done  10 out of  12 | elapsed: 86.8min remaining: 17.4min
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 104.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 104.3min finished


In [24]:
print(rf_model.cv_results_)
dump(rf_model, 'gs_rf.pkl')

{'mean_fit_time': array([ 773.5180583 , 1564.84641671, 3107.25946712,  782.37250566,
       1560.30513787, 3140.65509033,  779.82531023, 1552.67249751,
       2897.27446055,  780.50803185, 1320.76549625, 2366.28310227]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.16279006, 0.17701006, 0.17773414, 0.16427469, 0.16144609,
       0.18402052, 0.14855528, 0.18253398, 0.11143184, 0.16162205,
       0.10688281, 0.10416555]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1, 1, 1, 1, 1, 1],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[100, 100, 100, 1000, 1000, 1000, 100, 100, 100, 1000,
                   1000, 1000],
             mask=[False, False, False, False, False, Fal

['gs_rf.pkl']

In [ ]:
%reset_selective "^rf_model$"

In [20]:
xg_base = xgb.XGBClassifier()
xg_grid = {
"max_depth" : [100, 1000],
"learning_rate" : [0.1, 1],
"n_estimators" : [50, 100, 200]
}
xg_model = GridSearchCV(xg_base, xg_grid, cv=cv, n_jobs=4, verbose=100)

In [21]:
xg_model = xg_model.fit(X, y)

Fitting 1 folds for each of 12 candidates, totalling 12 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 15.1min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed: 15.3min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed: 25.6min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed: 32.5min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 40.7min
[Parallel(n_jobs=4)]: Done   6 out of  12 | elapsed: 40.7min remaining: 40.7min
[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 42.4min remaining: 30.3min
[Parallel(n_jobs=4)]: Done   8 out of  12 | elapsed: 47.5min remaining: 23.8min
[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 52.2min remaining: 17.4min
[Parallel(n_jobs=4)]: Done  10 out of  12 | elapsed: 54.3min remaining: 10.9min
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 58.9min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 58.9min finished


In [23]:
print(xg_model.cv_results_)
dump(xg_model, 'gs_xg.pkl')

{'mean_fit_time': array([ 913.71682143, 1535.35235405, 2441.25160146,  904.67709804,
       1535.48571825, 2395.25545144,  413.36639786,  590.16139865,
        816.5187192 ,  409.22546744,  589.20019341,  682.79986906]), 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'mean_score_time': array([0.1659472 , 0.15870285, 0.16429615, 0.15463281, 0.16236734,
       0.09843564, 0.15126419, 0.14911413, 0.15361142, 0.15044475,
       0.15081024, 0.0919559 ]), 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1, 1, 1, 1, 1, 1],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[100, 100, 100, 1000, 1000, 1000, 100, 100, 100, 1000,
                   1000, 1000],
             mask=[False, False, False, False, False, Fal

['gs_xg.pkl']